In [1]:
!pip install PyMuPDF

     ---------------------------------------- 16.5/16.5 MB 3.6 MB/s eta 0:00:00


In [3]:
import fitz  # PyMuPDF

# Open the PDF file
with fitz.open("The_Verdict.pdf") as doc:
    raw_text = ""
    # Loop through each page of the PDF and extract text
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)  # Load page by number
        raw_text += page.get_text()  # Extract text from page

print(raw_text)


1
The Verdict
Edith Wharton
1908
Exported from Wikisource on May 20, 2024
2
I HAD always thought Jack Gisburn rather a cheap genius--
though a
good fellow enough--so it was no great surprise to
me to hear that,
in the height of his glory, he had dropped
his painting, married a
rich widow, and established himself
in a villa on the Riviera.
(Though I rather thought it would
have been Rome or Florence.)
"The height of his glory"--that was what the women called
it. I can
hear Mrs. Gideon Thwing--his last Chicago sitter--
deploring his
 unaccountable abdication. "Of course it's
going to send the value of
my picture 'way up; but I don't
think of that, Mr. Rickham--the loss
to Arrt is all I think of."
The word, on Mrs. Thwing's lips,
 multiplied its _rs_ as
though they were reflected in an endless
vista of mirrors.
And it was not only the Mrs. Thwings who mourned.
Had
not the exquisite Hermia Croft, at the last Grafton Gallery
show, stopped me before Gisburn's "Moon-dancers" to say,
with tear

## Creating Tokens

In [4]:
## Use Regex function 

In [22]:
print("Total number of character:", len(raw_text))
print(raw_text[:100])

Total number of character: 21916
1
The Verdict
Edith Wharton
1908
Exported from Wikisource on May 20, 2024
2
I HAD always thought Jac


In [7]:
import re

text = "Hello, world. this is an example."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'this', ' ', 'is', ' ', 'an', ' ', 'example.']


In [8]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'this', ' ', 'is', ' ', 'an', ' ', 'example', '.', '']


In [9]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'this', 'is', 'an', 'example', '.']


In [10]:
text = "Hello, world. Is this :::: a () example?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', ':', ':', ':', ':', 'a', '(', ')', 'example', '?']


In [11]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', ':', ':', ':', ':', 'a', '(', ')', 'example', '?']


In [12]:
text = "Hello, world. Is this-()- an example?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this-', '(', ')', '-', 'an', 'example', '?']


In [14]:
pre_processed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
pre_processed = [item.strip() for item in pre_processed if item.strip()]
print(pre_processed[:10])
print(len(pre_processed))


['1', 'The', 'Verdict', 'Edith', 'Wharton', '1908', 'Exported', 'from', 'Wikisource', 'on']
4947


## Create Token IDS

In [33]:
words = sorted(set(pre_processed))
vocab_size = len(words)

print(vocab_size)

1252


In [34]:
vocab = {token:integer for integer,token in enumerate(words)}

In [35]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 60:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
('0', 8)
('1', 9)
('10', 10)
('11', 11)
('12', 12)
('13', 13)
('14', 14)
('15', 15)
('16', 16)
('17', 17)
('18', 18)
('19', 19)
('1908', 20)
('1929', 21)
('2', 22)
('20', 23)
('2024', 24)
('21', 25)
('22', 26)
('3', 27)
('4', 28)
('5', 29)
('6', 30)
('7', 31)
('8', 32)
('9', 33)
(':', 34)
(';', 35)
('?', 36)
('A', 37)
('Abigor', 38)
('About', 39)
('AdamBMorgan', 40)
('Ah', 41)
('Among', 42)
('And', 43)
('Are', 44)
('Arrt', 45)
('As', 46)
('At', 47)
('Attribution-ShareAlike', 48)
('AzaToth', 49)
('Be', 50)
('Begin', 51)
('Bender235', 52)
('Blurpeace', 53)
('Boris23', 54)
('Bromskloss', 55)
('Burlington', 56)
('But', 57)
('By', 58)
('Carlo', 59)
('Chicago', 60)


In [36]:
class TokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        pre_processed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        pre_processed = [
            item.strip() for item in pre_processed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in pre_processed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [37]:
tokenizer = TokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 102, 2, 962, 1104, 696, 626, 851, 5, 1248, 690, 5, 1, 117, 7, 83, 963, 1229, 859, 900, 7]


In [38]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [39]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

In [26]:
all_tokens = sorted(list(set(pre_processed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [27]:
len(vocab.items())

1254

In [28]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1249)
('your', 1250)
('yourself', 1251)
('<|endoftext|>', 1252)
('<|unk|>', 1253)


In [29]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [30]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [31]:
tokenizer.encode(text)

[1253,
 5,
 440,
 1248,
 725,
 1089,
 36,
 1252,
 100,
 1104,
 1070,
 1100,
 825,
 1104,
 1253,
 7]

In [32]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

### BYTE PAIR ENCODING (BPE)

In [40]:
! pip3 install tiktoken

     -------------------------------------- 894.0/894.0 kB 4.7 MB/s eta 0:00:00


In [42]:
import importlib
import tiktoken



In [43]:
tokenizer = tiktoken.get_encoding("gpt2")

In [47]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     " of snownPlaceomeunk."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [48]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


In [46]:
integers = tokenizer.encode("Akwirw ier")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier
